In [1]:
from PyBTLS.py_module.bridge.bridge import Bridge
from PyBTLS.py_module.bridge.flow import LaneFlow, BridgeFlow
from PyBTLS.py_module.parallel.parallel import BTLSWrapper
from PyBTLS.py_module.traffic.constant_vehicle import ConstantVehicle
from PyBTLS.py_module.traffic.garage import Garage, Kernels
from PyBTLS.py_module.traffic.PyBtlsDefaultTraffic import Auxerre #OLD CODE, BUT NO REPLACEMENT YET
import numpy as np

In [2]:
# Define the bridge
# This is similar to PyCBA syntax
L = [25.0] #m
R = [-1, 0, -1, 0] #Simply supported
bridge = Bridge(L=L, R=R, n_lane=1)

# Define the Load Effects to analyse
# Adding load effects can be done in two ways
# The first way is to use BTLS built-in influence line
# e.g., this is Influence Line 1, which computes moment at midspan
# See BTLS manual for full list of built-in load effects
bridge.add_load_effect(load_effect = 1) 
# Another way to define load effects is by specifiying:
#   load_effect ('M', 'V', or 'R' for moment, shear or reaction)
#   poi         (The point where the load effect is to be measured)
#   step        (The resolution of the IL calculation. Higher resolution is provides more accurare result but will slow down run time)
# Some other available arguments are:
#   lane_factor (The lane_factor. Can be a list for different factors for each lane, or a singular value for same lane_factor for all. default 1.0)
#   POT         (Minimum LE to be included in Peak Over Threshold (POT) analysis. default to 0.0)
bridge.add_load_effect(load_effect = "M", poi = 3/4 * L[0], step = 0.1)

# Defining the lane flow
# Lane flow can be constructed using:
#   - data frame (NOT recommended as it is prone to error)
#   - csv file
#   - default files
# Construction via data frame is done using the Pandas.DataFrame arguments and syntax. See Pandas documentation for details.
# Construction via csv file require you to supply the following arguments:
#   path        (Path to the csv file, in string)
# Construction via the default files can be done using the load_default() method. The method can take the argument:
#   sample      (The sample number, either 1 or 2. Currently there are two default lane flow definitions available. Default to 1)
lane_1 = LaneFlow()
lane_1.load_default()


# Define the Bridge Flows
# Multiple lanes can be created. This is then added to a BridgeFlow object, which describe the flow of all the lanes in the bridge
# Lane can be added at initialisation as list or BtlsFlow object, or after initialisation via:
#   - add() method or other aliases of the method: add_lane_flow(), append(), extend())
#   - insert(idx, lane_flow) : insert new lane flow into the array
#   - list indexing          : This is similar to regular list indexing, e.g. bridge_flow[1] = lane_2, del bridge_flow[0], etc.
bridge_flow = BridgeFlow(lane_1)

In [3]:
# DEVELOPMENT VERSION
# THIS WILL CHANGE IN THE FUTURE AS THE txt FILES ARE DEPRECATED

# Creating BTLS wrapper
# The bridge needs to then be wrapped in a BTLSWrapper class. This class provides the method of parsing the required input and reading outputs.
# Arguments here are:
# bridge        (A Bridge object)
# num_days      (Number of days for the simulation)
# working_dir   (The working directory where all txt files will be created and read)
btls = BTLSWrapper(bridge = bridge, bridge_flow = bridge_flow, num_days = 250)

# To change analysis types and other options, see the options object
# Set the Simulation mods:
btls.options.program_mode = "Gen & Sim"
# In this case, we're interested in block maxima analysis. We could also do interval statistics to get a quick overview later on
# We can invoke the setiings one by one, e.g.:
#btls.options.analyse_block_max = 1
#btls.options.write_block_max_summary_files = 1
#btls.options.write_block_max_mixed_analysis = 1
# A simpler way to do this is for a set of analysis, you could do:
btls.options.group_analyse_block_maxima = True
btls.options.group_analyse_statistics = True
# Or to turn off an analysis (by default all analysis are off)
btls.options.group_analyse_pot = False
#Other options, e.g. set simulation time step
btls.options.time_step = 0.01


# Define the traffic objects
# NEED WORK HERE
#Traffic Vehicle Settings
btls.options.vehicle_generation_model = "Constant"
btls.options.headway_model = "Free Flow"

constant_vehicle = ConstantVehicle()
constant_vehicle.load_default()
btls.constant_vehicle = constant_vehicle

garage = Garage()
garage.load_default()
btls.garage = garage

kernels = Kernels()
kernels.load_default()
btls.kernels = kernels

btls.traffic = Auxerre #OLD CODE, BUT NO REPLACEMENT YET

#Print the options. Can also use .print() method
print(btls.options)

PyBTLS BTLSWrapperOptions object at 0x7ff4fc8e4370
{
  program_mode: Gen & Sim,
  num_days: 250,
  track_width: 190.0,
  eccentricity_std_dev: 20,
  vehicle_generation_model: Constant,
  headway_model: Free Flow,
  traffic_classification: 1,
  nom_congested_spacing: 26.1,
  congested_speed: 49.7,
  congested_gaps_cov: 0.05,

  impose_constant_speed: 0,
  use_avg_speed_if_impose_constant_speed: 1,
  constant_speed_if_not_use_avg_speed: 88,

  time_step: 0.01,
  min_GVW: 0.0,

  write_full_time_history: 0,
  write_each_loading_event: 0,
  write_each_loading_event_buffer_size: 10000000,
  write_fatigue_event: 0,
  write_rainflow_algorithm: 0,
  num_decimal_rainflow_algorithm: 3,
  fatigue_cutoff_value: 5.0,

  write_vehicle_file: 0,
  write_vehicle_file_format: MON,
  write_vehicle_file_buffer_size: 1000000,
  write_vehicle_flow_statistics: 0,

  analyse_block_max: 1,
  size_block_max_in_days: 1,
  size_block_max_in_seconds: 0,
  write_block_max_separated_vehicle_files: 1,
  write_block_m

In [4]:
# We can then run the analysis. Output will be read automatically and the working directory will be purged
btls.run()

# Save the object so we don't lose our results!
fname = 'Example 1 Bridge Block Maxima Analysis.obj'
btls.save(fname)

# We can reload it in the future using the load() method
btls = BTLSWrapper()
btls.load(fname)

---------------------------------------------
Bridge Traffic Load Simulation - C.C. Caprani
                Version 1.3.5			      
---------------------------------------------

Program Mode: 1

Reading influence surface file: ./Input/IS.txt
Reading influence line file: ./Input/IL.txt
Starting simulation...
Day complete...


*** Warning: Influence Surface file could not be opened


1	2	3	4	5	6	7	8	9	10
11	12	13	14	15	16	17	18	19	20
21	22	23	24	25	26	27	28	29	30
31	32	33	34	35	36	37	38	39	40
41	42	43	44	45	46	47	48	49	50
51	52	53	54	55	56	57	58	59	60
61	62	63	64	65	66	67	68	69	70
71	72	73	74	75	76	77	78	79	80
81	82	83	84	85	86	87	88	89	90
91	92	93	94	95	96	97	98	99	100
101	102	103	104	105	106	107	108	109	110
111	112	113	114	115	116	117	118	119	120
121	122	123	124	125	126	127	128	129	130
131	132	133	134	135	136	137	138	139	140
141	142	143	144	145	146	147	148	149	150
151	152	153	154	155	156	157	158	159	160
161	162	163	164	165	166	167	168	169	170
171	172	173	174	175	176	177	178	179	180
181	182	183	184	185	186	187	188	189	190
191	192	193	194	195	196	197	198	199	200
201	202	203	204	205	206	207	208	209	210
211	212	213	214	215	216	217	218	219	220
221	222	223	224	225	226	227	228	229	230
231	232	233	234	235	236	237	238	239	240
241	242	243	244	245	246	247	248	249	250


Simulation complete

Duration of analysis: 175.454 s


NameError: name 'PyBtlsOutput' is not defined

In [8]:
# Define Parallelisation
# The above BTLSWrapper can be run using the run() method. But

In [6]:
btls.working_dir

'.btls_default_working_dir'

In [2]:
Auxerre